In [16]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

# Load the training dataset
train_df = pd.read_csv('train_preprocessed.csv')

# Handle missing values in the training dataset
train_df.fillna("", inplace=True)

# Define the features and labels for the training dataset
X_train = train_df['comment_text']
y_train = train_df[['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic']]

# Preprocess the training text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)

# Load the testing dataset
test_df = pd.read_csv('test_preprocessed.csv')

# Handle missing values in the testing dataset
test_df.fillna("", inplace=True)

# Define the features for the testing dataset
X_test = test_df['comment_text']

# Preprocess the test text data using TF-IDF
X_test_vec = vectorizer.transform(X_test)

# Define the pipeline
pipeline = Pipeline([
    ('clf', MultiOutputClassifier(LogisticRegression()))
])

In [17]:
# Train the model
pipeline.fit(X_train_vec, y_train)


Pipeline(steps=[('clf', MultiOutputClassifier(estimator=LogisticRegression()))])

In [80]:
# Predict on the test data
y_test_pred = pipeline.predict(X_test_vec)

# Convert the predictions to a DataFrame for easier handling
y_pred_df = pd.DataFrame(y_test_pred, columns=y_train.columns)

# Display the predictions
print(y_pred_df)

        identity_hate  insult  obscene  severe_toxic  threat  toxic
0                   0       1        1             0       0      1
1                   0       0        0             0       0      0
2                   0       0        0             0       0      0
3                   0       0        0             0       0      0
4                   0       0        0             0       0      0
...               ...     ...      ...           ...     ...    ...
153159              0       0        0             0       0      1
153160              0       0        0             0       0      0
153161              0       0        0             0       0      0
153162              0       0        0             0       0      0
153163              0       1        1             0       0      1

[153164 rows x 6 columns]


In [79]:
# Define a new comment
new_comment = ["fuck you you fucking"]

# Transform the comment using the TfidfVectorizer
new_comment_vec = vectorizer.transform(new_comment)

# Use the trained model to predict the labels for the comment
new_comment_pred = pipeline.predict(new_comment_vec)

# Define the labels
labels = ['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic']

# Format and print the predictions
for i, label in enumerate(labels):
    print(f"{label}: {new_comment_pred[0][i]}")




identity_hate: 0
insult: 1
obscene: 1
severe_toxic: 1
threat: 0
toxic: 1


In [89]:
# Define a new comment
new_comment = ["Hello guys today I'm going to kill you i hope your family gets cancer and dies and you live alone you piece of shit suck my cock bitch"]

# Transform the comment using the TfidfVectorizer
new_comment_vec = vectorizer.transform(new_comment)

# Use the trained model to predict the labels for the comment
new_comment_pred = pipeline.predict(new_comment_vec)

# Remove the 'toxic' label
new_comment_pred_without_toxic = new_comment_pred[0][:-1]

# Define weights for the remaining labels
weights = [3, 1, 1, 3, 3]  # 'identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat'

# Check if there is at least one label with a prediction of 1
has_toxic_label = any(label_pred == 1 for label_pred in new_comment_pred_without_toxic)

if has_toxic_label:
    # make sure the toxicity score is at least 0.6
    # also the np.average does not go over 0.6
    toxicity_score = 0.6 + np.average(new_comment_pred_without_toxic, weights=weights) * 0.4
else:
    toxicity_score = 0


# Format and print the predictions
for i, label in enumerate(labels):
    print(f"{label}: {new_comment_pred[0][i]}")

# Print the toxicity score
print(toxicity_score)


identity_hate: 0
insult: 1
obscene: 1
severe_toxic: 1
threat: 1
toxic: 1
0.8909090909090909
